# OpenPoseを使ってパラメータ取得

## モジュール読み込み

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from tqdm import tqdm as tqdm

import os
import sys
sys.path.append("../")
from utils import variables

In [4]:
from openpose.openpose import OpenPose

## 1枚の静止画に対してパラメータを取得

In [5]:
cap = cv2.VideoCapture("/media/nvidia/EndoLab-3TB/s.tawada/AbemaTVトーナメント予選_高見久保戦.mp4")

In [6]:
params = variables.params

In [7]:
openpose = OpenPose(params)

|parts|||||
|---|||||
|0: "Nose"|1: "Neck"|2: "RShoulder"|3: "RElbow"|4: "RWrist",|5: "LShoulder"|
|6: "LElbow"|7: "LWrist"|8: "MidHip"|9: "RHip"|10:"RKnee"|
|11:"RAnkle"|12:"LHip"|13:"LKnee"|14:"LAnkle"|15:"REye"|
|16:"LEye"|17:"REar"|18:"LEar"|19:"LBigToe"|20:"LSmallToe"|
|21:"LHeel"|22:"RBigToe"|23:"RSmallToe"|24:"RHeel"|25:"Background"|

今回取得するのはNose,Neck,RShoulder,RElbow, LShoulder, LElbow, REye, LEye

In [8]:
keypoints_dict = variables.keypoints_dict
get_parts_list = ["Nose","Neck","RShoulder","RElbow", "LShoulder", "LElbow", "REye", "LEye"]

## フレームごとにサンプリングしOpenPoseを適用

In [10]:
# opencv * multiprocessing
from multiprocessing import Pool, Manager
import multiprocessing as multi
from time import time

n_frame = 30
output_path = "/media/nvidia/EndoLab-3TB/s.tawada/Research/output/"

# 動画のサンプリングフレームのインデックスを取得
cap = cv2.VideoCapture("/media/nvidia/EndoLab-3TB/s.tawada/AbemaTVトーナメント予選_高見久保戦.mp4")
frame_all = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_idx = [_ for _ in range(frame_all) if _%n_frame == 0]

def get_features(idx):
    print("Processing to {}".format(idx))
    cap = cv2.VideoCapture("/media/nvidia/EndoLab-3TB/s.tawada/AbemaTVトーナメント予選_高見久保戦.mp4")
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    _, frame = cap.read()

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.rectangle(img, (0, 0), (600, 100), (0, 0, 0), -1)
    
    keypoints, output_image = openpose.forward(img, True)
    
    if len(keypoints) == 2:
        player1.update({idx:keypoint[0][[keypoints_dict[i] for i in get_parts_list]].ravel()})
        player2.update({idx:keypoint[1][[keypoints_dict[i] for i in get_parts_list]].ravel()})

    cv2.imwrite(os.path.join(output_path, "img/{0:04d}.png".format(counter)), cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))
    
player1 = Manager().dict()
player2 = Manager().dict()

tic = time()
cv2.setNumThreads(0)
with Pool(2) as p:
    p.map(get_features, frame_idx[:5], 1)
toc = time()
print("Execute time: {}s".format(toc - tic))

p1_df = pd.DataFrame(np.array([_ for _ in player1.values()]), \
        columns=["{}{}".format(parts, point) for parts in get_parts_list for point in ["_x", "_y", "_z"]])
p2_df = pd.DataFrame(np.array([_ for _ in player2.values()]), \
        columns=["{}{}".format(parts, point) for parts in get_parts_list for point in ["_x", "_y", "_z"]])

p1_df.to_csv(os.path.join(output_path, "player1_features.csv"))
p2_df.to_csv(os.path.join(output_path, "player2_features.csv"))

Processing to 0
Processing to 30
Processing to 60
Processing to 90
Processing to 120
Execute time: 0.8674538135528564s


ValueError: Empty data passed with indices specified.